In [23]:
import pandas as pd
from collections import Counter
import numpy as np
import glob
import re


In [24]:
licenses = pd.read_csv("..//data//searchResults.csv")
weedmaps = pd.read_csv("..//data//store.csv")

In [25]:
check_cols = ["adult_use_cultivation", "adult_use_nonstorefront", 
              "adult_use_retail", "distributor", "medical_cultivation", 
              "medical_nonstorefront", "medical_retail", "microbusiness"]

In [26]:
weedmaps["adult_use_cultivation"].dtype == "float"#.str.upper()

True

In [ ]:
cleanString = re.sub('\W+','', string )

In [66]:
for col in check_cols:
    if weedmaps[col].dtype != "float":
        weedmaps[col] = weedmaps[col].str.upper()
        weedmaps[col] = weedmaps[col].str.replace('-', '')

In [67]:
licenses['License Number'] = licenses['License Number'].str.replace('-', '')

In [55]:
merge = pd.merge(weedmaps, 
                 licenses, 
                 left_on = 'adult_use_nonstorefront',
                 right_on = "License Number",
                 how = "outer",
                 indicator = True)
Counter(merge['_merge'])

Counter({'both': 222, 'left_only': 1276, 'right_only': 3396})

In [68]:
for i in check_cols:
    try:
        merge = pd.merge(weedmaps, 
                         licenses, 
                         left_on = i,
                         right_on = "License Number",
                         how = "outer",
                         indicator = True
        )
        print(Counter(merge["_merge"]), i)
    except:
        print("failed", i)

failed adult_use_cultivation
Counter({'right_only': 3393, 'left_only': 1251, 'both': 247}) adult_use_nonstorefront
Counter({'right_only': 3286, 'left_only': 1300, 'both': 198}) adult_use_retail
Counter({'right_only': 3450, 'left_only': 1489, 'both': 9}) distributor
failed medical_cultivation
Counter({'right_only': 3416, 'left_only': 1362, 'both': 136}) medical_nonstorefront
Counter({'right_only': 3375, 'left_only': 1409, 'both': 89}) medical_retail
Counter({'right_only': 3435, 'left_only': 1472, 'both': 26}) microbusiness


In [37]:
for i in check_cols:
    try:
        merge = pd.merge(weedmaps, 
                         licenses, 
                         left_on = i,
                         right_on = "License Number",
                         how = "outer",
                         indicator = True
        )
        print(Counter(merge["_merge"])["both"], i)
        stats = Counter(merge[merge["_merge"] == 'both']["License Type"])
        for x in stats:
            print(x, stats[x])
        print()
    except:
        print("failed", i)

failed adult_use_cultivation
222 adult_use_nonstorefront
Cannabis - Retailer Nonstorefront Temporary License 182
Cannabis - Retailer Temporary License 40

181 adult_use_retail
Cannabis - Retailer Temporary License 163
Cannabis - Retailer License 6
Cannabis - Retailer Nonstorefront Temporary License 7
Cannabis - Microbusiness Temporary License 4
Cannabis - Distributor Temporary License 1

1 distributor
Cannabis - Distributor Temporary License 1

failed medical_cultivation
109 medical_nonstorefront
Cannabis - Retailer Nonstorefront Temporary License 104
Cannabis - Retailer Temporary License 5

80 medical_retail
Cannabis - Retailer Temporary License 68
Cannabis - Retailer License 6
Cannabis - Microbusiness Temporary License 1
Cannabis - Retailer Nonstorefront Temporary License 4
Cannabis - Distributor Temporary License 1

25 microbusiness
Cannabis - Microbusiness Temporary License 25



In [69]:
'''
Creates a new dataframe (weedmaps_joined) which contains the joins of weedmaps stores
that had a license number in the licenses dataframe. The license numbers are joined in 
the order that they appear on check_cols.

Create another dataframe (weedmaps_not_joined) which contains all the weedmap stores that
had licenses that could not be joined to the licenses dataframe. 
'''

weedmaps_not_joined = weedmaps
weedmaps_joined = pd.DataFrame()

for i in check_cols:
    try:
        join_on_i = pd.merge(weedmaps_not_joined, licenses, left_on = i, right_on = 'License Number', how = 'inner')
        
        print ("joined:", join_on_i.shape[0], ' on:', i)
        weedmaps_not_joined = weedmaps_not_joined[~weedmaps_not_joined[i].isin(join_on_i['License Number'])]
        weedmaps_joined = pd.concat([weedmaps_joined, join_on_i])
        
    except:
        print(i)

weedmaps_not_joined.to_csv('..//data//weedmaps_not_joined.csv')
weedmaps_joined.to_csv('..//data//weedmaps_joined.csv')

adult_use_cultivation
joined: 247  on: adult_use_nonstorefront
joined: 197  on: adult_use_retail
joined: 0  on: distributor
medical_cultivation
joined: 11  on: medical_nonstorefront
joined: 42  on: medical_retail
joined: 25  on: microbusiness


In [77]:
weedmaps_not_joined.head(100)

,Unnamed: 0,id,name,state,city,latitude,longitude,license_type,address,rating,...,retailer_services,phone,adult_use_cultivation,adult_use_nonstorefront,adult_use_retail,distributor,medical_cultivation,medical_nonstorefront,medical_retail,microbusiness
11,11,50268,Lytt,CA,Pacifica,37.634507,-122.491800,medical,1905 Palmetto Ave. Unit B,4.0,...,storefront,0000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,60567,Harvest of Napa (Newly Opened),CA,Napa,38.297243,-122.301418,medical,2449 Second Street,3.0,...,storefront,707.210.2271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,15,49957,Mendocino Organics,CA,Vallejo,38.109695,-122.253425,medical,539 A Tennessee ST,3.0,...,storefront,(707)652-5018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,16,20208,Fly Fast Delivery - Richmond,CA,Richmond,37.964331,-122.318325,medical,NaN,4.0,...,delivery,(510) 776-2965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,57685,East Bay Therapeutics,CA,Emeryville,37.838307,-122.290292,hybrid,5745 Peladeau Street,4.0,...,storefront,(510) 925-1266,NaN,NaN,A100000368,NaN,NaN,NaN,M100000420,NaN
23,23,52646,Phog Center,CA,Pacifica,37.631547,-122.492719,medical,2270 Palmetto Ave,4.0,...,storefront,844-504-8771,NaN,NaN,NaN,NaN,NaN,NaN,C1018000012010,NaN
27,27,54343,ME Dispensary,CA,El Sobrante,37.965296,-122.314280,medical,3823 San Pablo Dam Rd,3.0,...,storefront,510-758-3269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,28,3509,E-leaf,CA,San Rafael,37.974566,-122.538539,medical,NaN,4.0,...,delivery,415-943-1420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,32,3136,Purple Buddha - Vallejo,CA,Vallejo,38.104806,-122.243560,medical,NaN,3.0,...,delivery,925-993-8180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,33,38967,MINT XPRESS - S. San Francisco,CA,San Francisco,37.656922,-122.454226,hybrid,NaN,3.0,...,delivery,(650) 285-1601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
A100000368

In [90]:
licenses[licenses['License Number'].str.contains('420')]

,License Number,License Type,Business Owner,Business Contact Information,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal
1147,C11180000420,Cannabis - Distributor Temporary License,Brian McCloud,GREENEYES FARMS INC : Email- bmcdrywall@yahoo....,Corporation,"CHATSWORTH, CA 913114809 County: LOS ANGELES",Active,07/09/2019,12/11/2018,N/A for this license type,BOTH
